In [1]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Dostoevsky

Информация

In [4]:
pip install dostoevsky

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.8 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=3133200 sha256=6345ab9fc7d2d3f8cad0da8266fa5c97398391842c1301b7aaa0440b2abe6bb1
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


In [5]:
!python -m dostoevsky download fasttext-social-network-model

In [6]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
import fasttext
fasttext.FastText.eprint = lambda x: None

tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

neutral | positive | negative | skip | speech

# Подготовка 

In [7]:
def convert(row, is_num=False):
  # print(row['text'])
  dict = model.predict(row['text'], k=5)[0]
  # print(model.predict(row['text'], k=5))
  # print(dict)
  str = max(dict, key=dict.get) # smth with max value of 'confidence'

  if str == 'positive':
    if is_num:
      dict['pred'] = 1
    else:
      dict['pred'] = 'PSTV' 
  elif str == 'negative':
    if is_num:
      dict['pred'] = -1
    else:
      dict['pred'] = 'NGTV' 
  else:
    if is_num:
      dict['pred'] = 0
    else:
      dict['pred'] = 'NEUT' 
  
  return dict['pred']

  # for key, value in dict.items():
  #   row[key] = value

In [ ]:
df = pd.read_csv('drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/train.csv', sep=',', nrows=10, on_bad_lines='skip').drop('Unnamed: 0', axis=1)
df['pred'] = 0
df['test'] = 0
df

,text,sentiment,pred,test
0,".с.,и спросил его: о Посланник Аллаха!Ты пори...",1,0,0
1,Роднее всех родных Попала я в ГКБ №8 еще в дек...,1,0,0
2,Непорядочное отношение к своим работникам Рабо...,2,0,0
3,"). Отсутствуют нормативы, Госты и прочее, что ...",1,0,0
4,У меня машина в руках 5 лет и это п...,1,0,0
5,Самое лучшее в отеле - место его расположения ...,0,0,0
6,Дорогая аптека Дорогая аптека. В других сетях ...,1,0,0
7,Останавливались с женой на годовщину. Отель по...,1,0,0
8,"Персонал вежливый, все сделали аккуратно В цел...",1,0,0
9,Второй раз останавливаюсь в этом отеле. С перв...,0,0,0


In [ ]:
df.apply(lambda x: convert(x), axis=1)
df

,text,sentiment,pred,test
0,".с.,и спросил его: о Посланник Аллаха!Ты пори...",1,0,0
1,Роднее всех родных Попала я в ГКБ №8 еще в дек...,1,0,0
2,Непорядочное отношение к своим работникам Рабо...,2,0,0
3,"). Отсутствуют нормативы, Госты и прочее, что ...",1,0,0
4,У меня машина в руках 5 лет и это п...,1,0,0
5,Самое лучшее в отеле - место его расположения ...,0,0,0
6,Дорогая аптека Дорогая аптека. В других сетях ...,1,0,0
7,Останавливались с женой на годовщину. Отель по...,1,0,0
8,"Персонал вежливый, все сделали аккуратно В цел...",1,0,0
9,Второй раз останавливаюсь в этом отеле. С перв...,0,0,0


In [ ]:
ab = []
for i, v in df.iterrows():
  print(v)
  v['pred'] = convert(v)

text         .с.,и спросил его:  о Посланник Аллаха!Ты пори...
sentiment                                                    1
pred                                                         0
test                                                         0
Name: 0, dtype: object
text         Роднее всех родных Попала я в ГКБ №8 еще в дек...
sentiment                                                    1
pred                                                         0
test                                                         0
Name: 1, dtype: object
text         Непорядочное отношение к своим работникам Рабо...
sentiment                                                    2
pred                                                         0
test                                                         0
Name: 2, dtype: object
text         ). Отсутствуют нормативы, Госты и прочее, что ...
sentiment                                                    1
pred                                             

In [ ]:
texts = df['text']
results = model.predict(texts, k=5)

ab = []
for text, sentiment in zip(texts, results):
  print(sentiment, ' <- ', text)

  str = max(sentiment, key=sentiment.get) # smth with max value of 'confidence'
  if str == 'positive':
    ab.append('PSTV')
  elif str == 'negative':
    ab.append('NGTV')
  else:
    ab.append('NEUT')

df['pred'] = ab

{'neutral': 0.46102678775787354, 'negative': 0.22271016240119934, 'skip': 0.0980893224477768, 'positive': 0.06755668669939041, 'speech': 0.029322227463126183}  <-  .с.,и спросил его:  о Посланник Аллаха!Ты порицаешь что-то из слушания?  Он ответил: я не порицаю ничего из него,но передай им,чтобы они открывали свои собрания Кораном и закрывали их Кораном ...........Это дошедшие до нас мнения и тот кто находится в поисках истины,по мере изучения этого вопроса будет сталкиваться с разногласиями и будет оставаться в растерянности или склонится к мнению одной из сторон по своему желанию.Но всего этого недостаточно,потому что он сам должен найти истину,подробно изучив вопросы запретного и разрешённого.|||||||||||||||||||||||||||||||||||||Обрати внимание:основатели всех четырёх мазхабов осуждали песни и поэззию.И этим часто грешат заблудшие суфии.Исключение делается для исламской тематики  Сахих БухариО проявлении радости во время праздника.498 (949). Сообщается, что ‘Аиша, да будет доволен е

In [ ]:
dict = {'neutral': 0.4301573634147644, 'positive': 0.21207880973815918, 'negative': 0.1067005917429924, 'skip': 0.04743587225675583, 'speech': 0.01799621433019638}
dict['pred'] = 'PSTV'
dict

{'neutral': 0.4301573634147644,
 'positive': 0.21207880973815918,
 'negative': 0.1067005917429924,
 'skip': 0.04743587225675583,
 'speech': 0.01799621433019638,
 'pred': 'PSTV'}

# F1-мера

In [12]:
from sklearn.metrics import precision_recall_fscore_support as alotmetrics

In [ ]:
test = pd.read_csv('drive/MyDrive/ОБЩЕЕ/РСОДПО/AI/emotional/data/train.csv', sep=',', nrows=10000, on_bad_lines='skip').drop('Unnamed: 0', axis=1)
test.columns = ['text', 'true']
test

,text,true
0,".с.,и спросил его: о Посланник Аллаха!Ты пори...",1
1,Роднее всех родных Попала я в ГКБ №8 еще в дек...,1
2,Непорядочное отношение к своим работникам Рабо...,2
3,"). Отсутствуют нормативы, Госты и прочее, что ...",1
4,У меня машина в руках 5 лет и это п...,1
...,...,...
9995,Огромное спасибо врачам и медсёстрам 6-го акуш...,1
9996,"Заведение приличное , только пить пиво нельзя ...",0
9997,"Сейчас мы тут, номер 7108 с видом на дорогу) з...",1
9998,"Врач Коваленченко, грубое отношение к пациента...",2


In [ ]:
# test = test.replace({'true' : { 2 : -1, 0 : 0, 1 : 1}})
# NEUT, PSTV, NGTV -> NGTV, NEUT, PSTV
test = test.replace({'true' : { 2 : 'NGTV', 0 : 'NEUT', 1 : 'PSTV'}})
test

,text,true
0,".с.,и спросил его: о Посланник Аллаха!Ты пори...",PSTV
1,Роднее всех родных Попала я в ГКБ №8 еще в дек...,PSTV
2,Непорядочное отношение к своим работникам Рабо...,NGTV
3,"). Отсутствуют нормативы, Госты и прочее, что ...",PSTV
4,У меня машина в руках 5 лет и это п...,PSTV
...,...,...
9995,Огромное спасибо врачам и медсёстрам 6-го акуш...,PSTV
9996,"Заведение приличное , только пить пиво нельзя ...",NEUT
9997,"Сейчас мы тут, номер 7108 с видом на дорогу) з...",PSTV
9998,"Врач Коваленченко, грубое отношение к пациента...",NGTV


In [ ]:
%%time
texts = test['text']
results = model.predict(texts, k=5)

temp = []
for text, sentiment in zip(texts, results):
  # print(sentiment, ' <- ', text)

  str = max(sentiment, key=sentiment.get) # smth with max value of 'confidence'
  if str == 'positive':
    temp.append('PSTV')
  elif str == 'negative':
    temp.append('NGTV')
  else:
    temp.append('NEUT')

test['pred'] = temp

CPU times: user 17.5 s, sys: 129 ms, total: 17.6 s
Wall time: 19.3 s


In [ ]:
test

,text,true,pred
0,".с.,и спросил его: о Посланник Аллаха!Ты пори...",PSTV,NEUT
1,Роднее всех родных Попала я в ГКБ №8 еще в дек...,PSTV,NGTV
2,Непорядочное отношение к своим работникам Рабо...,NGTV,NEUT
3,"). Отсутствуют нормативы, Госты и прочее, что ...",PSTV,NEUT
4,У меня машина в руках 5 лет и это п...,PSTV,NGTV
...,...,...,...
9995,Огромное спасибо врачам и медсёстрам 6-го акуш...,PSTV,NEUT
9996,"Заведение приличное , только пить пиво нельзя ...",NEUT,NGTV
9997,"Сейчас мы тут, номер 7108 с видом на дорогу) з...",PSTV,NEUT
9998,"Врач Коваленченко, грубое отношение к пациента...",NGTV,NEUT


In [ ]:
print('Precision', 'Recall', 'F-score', 'HZ')
print(alotmetrics(test['true'], test['pred'], average='macro'))
print(alotmetrics(test['true'], test['pred'], average='micro'))
print(alotmetrics(test['true'], test['pred'], average='weighted'))

Precision Recall F-score HZ
(0.458176915135041, 0.43004213922203166, 0.3871311406314308, None)
(0.4016, 0.4016, 0.4016, None)
(0.5099456301800855, 0.4016, 0.3940344978655581, None)


# "Эмпирическая" мера

In [39]:
of = pd.read_csv('https://docs.google.com/spreadsheets/d/1kD4JRC_nDcF2ysmXKgYTA6jNpUi3vdbSmJY99lJxZis/export?format=csv')[['text', 'y', 'y_alter']].replace({'y' : { -1 : 'NGTV', 0 : 'NEUT', 1 : 'PSTV'}}).replace({'y_alter' : { -1 : 'NGTV', 0 : 'NEUT', 1 : 'PSTV'}})
of.columns = ['text', 'true', 'possible']
of

,text,true,possible
0,Электив сам по себе скучноватый.,NGTV,NEUT
1,"Я не считаю, что полученные знания помогут мне...",NGTV,NaN
2,Преподаватель очень тихо ведёт лекцию.,NEUT,NaN
3,Преподаватель не пытается как-то ухудшить проц...,NEUT,NaN
4,"Ничего не делаем, в конце 2 проекта, можно бра...",NEUT,PSTV
...,...,...,...
138,Такая теория никак не поможет ребятам заинтере...,NGTV,NaN
139,"Было бы гораздо лучше, будь преподаватель с ка...",NGTV,NEUT
140,Или это преподавали бы люди с кафедры предприн...,NEUT,NaN
141,"Толку учить пользоваться программой, где самый...",NGTV,NaN


In [40]:
%%time 
temp = []
for sentiment in model.predict(of['text'], k=5):

  str = max(sentiment, key=sentiment.get) # smth with max value of 'confidence'
  if str == 'positive':
    temp.append('PSTV')
  elif str == 'negative':
    temp.append('NGTV')
  else:
    temp.append('NEUT')

of['pred'] = temp

CPU times: user 89.7 ms, sys: 0 ns, total: 89.7 ms
Wall time: 90.9 ms


In [41]:
of

,text,true,possible,pred
0,Электив сам по себе скучноватый.,NGTV,NEUT,NEUT
1,"Я не считаю, что полученные знания помогут мне...",NGTV,NaN,NEUT
2,Преподаватель очень тихо ведёт лекцию.,NEUT,NaN,NEUT
3,Преподаватель не пытается как-то ухудшить проц...,NEUT,NaN,NEUT
4,"Ничего не делаем, в конце 2 проекта, можно бра...",NEUT,PSTV,NEUT
...,...,...,...,...
138,Такая теория никак не поможет ребятам заинтере...,NGTV,NaN,NEUT
139,"Было бы гораздо лучше, будь преподаватель с ка...",NGTV,NEUT,NEUT
140,Или это преподавали бы люди с кафедры предприн...,NEUT,NaN,NEUT
141,"Толку учить пользоваться программой, где самый...",NGTV,NaN,NEUT


In [42]:
from sklearn.metrics import precision_recall_fscore_support as alotmetrics

def custom_f1(y_pred, y_true, y_possible=None, mode='fact', average='macro'):
  y_min = []
  y_max = []
  for i in range(len(y_pred)):
    if (not(pd.isnull(y_possible[i]))):
      if (y_pred[i] == y_true[i]):
        y_min.append(y_possible[i])
        y_max.append(y_true[i])
      else:
        y_min.append(y_true[i])
        y_max.append(y_possible[i])
    else:
      y_max.append(y_true[i])
      y_min.append(y_true[i])

  if (mode == 'fact'):
    return alotmetrics(y_true, y_pred, average=average)[:3]
  elif (mode == 'min'):
    return alotmetrics(y_min, y_pred, average=average)[:3]
  elif (mode == 'max'):
    return alotmetrics(y_max, y_pred, average=average)[:3]
  else:
    print("Допустимые режимы: min/fact/max")
    return

In [44]:
print("Min: {}".format(custom_f1(of['pred'], of['true'], of['possible'], 'min')))
print("Fact: {}".format(custom_f1(of['pred'], of['true'], of['possible'], 'fact')))
print("Max: {}".format(custom_f1(of['pred'], of['true'], of['possible'], 'max')))

(0.5022419513944937, 0.3940058479532164, 0.32867469879518074)
(0.5333153977221774, 0.41551345962113667, 0.37585482036276)
(0.6127253161151467, 0.46351046698872783, 0.4573464912280702)


In [14]:
print('Precision', 'Recall', 'F-score')
print(alotmetrics(of['true'], of['pred'], average='macro')[:3])
print(alotmetrics(of['true'], of['pred'], average='micro')[:3])
print(alotmetrics(of['true'], of['pred'], average='weighted')[:3])

Precision Recall F-score
(0.5333153977221774, 0.41551345962113667, 0.37585482036276)
(0.4825174825174825, 0.4825174825174825, 0.4825174825174825)
(0.5397380397380397, 0.4825174825174825, 0.41557056047730373)
